In [1]:
# 코드 7-3 라이브러리 호출
import os  # 운영 체제의 모듈을 가져옵니다.

os.environ["TF_CPP_MIN_LOG_LEVEL"] = (
    "2"  # 케라스에서 발생하는 경고(warning) 메시지를 제거합니다.
)

import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

2024-12-03 01:39:13.879335: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733157553.893165  104674 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733157553.897595  104674 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# 코드 7-4 값 초기화
tf.random.set_seed(22)
np.random.seed(22)
assert tf.__version__.startswith("2.")  # 텐서플로 버전이 2임을 확인

batch_size = 128
total_words = 10000
max_review_len = 80
embedding_len = 100

In [3]:
# 코드 7-5 데이터셋 준비
# IMDB 데이터셋을 로드하고, 단어 수를 제한합니다.
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(
    num_words=total_words
)
# 시퀀스 길이를 max_review_len으로 패딩합니다.
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_review_len)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_review_len)

# 훈련 데이터셋을 변환합니다.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(10000).batch(batch_size, drop_remainder=True)
# 테스트 데이터셋을 변환합니다.
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_data = test_data.batch(batch_size, drop_remainder=True)

# 데이터셋의 형태를 출력합니다.
print("x_train_shape:", x_train.shape, tf.reduce_max(y_train), tf.reduce_min(y_train))
print("x_test_shape:", x_test.shape)

# 테스트 데이터셋의 샘플을 출력합니다.
sample = next(iter(test_data))
print(sample[0].shape)

x_train_shape: (25000, 80) tf.Tensor(1, shape=(), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
x_test_shape: (25000, 80)
(128, 80)


W0000 00:00:1733157565.993213  104674 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
# 코드 7-6 RNN 셀을 이용한 네트워크 생성
class RNN_Build(tf.keras.Model):
    def __init__(self, units):
        super(RNN_Build, self).__init__()

        self.units = units
        self.embedding = layers.Embedding(
            total_words, embedding_len, input_length=max_review_len
        )
        self.rnn_cell0 = layers.SimpleRNNCell(units, dropout=0.2)
        self.rnn_cell1 = layers.SimpleRNNCell(units, dropout=0.2)
        self.outlayer = layers.Dense(1)

    @tf.function
    def call(self, inputs, training=False):
        x = self.embedding(inputs)  # 입력 데이터에 원-핫 인코딩 적용
        state0 = tf.zeros([inputs.shape[0], self.units])
        state1 = tf.zeros([inputs.shape[0], self.units])
        for word in tf.unstack(x, axis=1):
            # out0, state0 각각에 self.RNNCell0에서 받아 온 값을 저장
            out0, state0 = self.rnn_cell0(word, [state0], training=training)
            # out1, state1 각각에 self.RNNCell1에서 받아 온 값을 저장
            out1, state1 = self.rnn_cell1(out0, [state1], training=training)
        # 출력층 out1을 적용한 후 그 값을 x 변수에 저장
        x = self.outlayer(out1)
        # 마지막으로 x에 시그모이드 활성화 함수를 적용하여 prob에 저장
        prob = tf.sigmoid(x)
        return prob
        # return tf.squeeze(prob)  # prob 반환

In [5]:
# 코드 7-7 모델 훈련
import time

units = 64
epochs = 4
t0 = time.time()  # 모형의 실행 시간을 위해 시간을 t0에 저장

model = RNN_Build(units)
model.compile(
    optimizer=Adam(0.001),
    loss=tf.losses.BinaryCrossentropy(from_logits=False),
    metrics=["accuracy"],
    # experimental_run_tf_function=False,
)

model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

Epoch 1/4


/home/halozhan/Study/.venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(128,), output.shape=(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 128, 1)

In [6]:
# RNN 셀 대신 RNN 레이어를 사용한 코드

# 코드 7-6 RNN 셀을 이용한 네트워크 생성
class RNN_Build(tf.keras.Model):
    def __init__(self, units):
        super(RNN_Build, self).__init__()

        self.units = units
        self.embedding = layers.Embedding(total_words, embedding_len, input_length=max_review_len)
        self.rnn = layers.SimpleRNN(units, return_sequences=False, return_state=False, dropout=0.2)
        self.outlayer = layers.Dense(1)

    def call(self, inputs, training=False):
        x = self.embedding(inputs)
        x = self.rnn(x, training=training)
        x = self.outlayer(x)
        return x

# 코드 7-7 모델 훈련
import time

units = 64
epochs = 4
t0 = time.time()  # 모형의 실행 시간을 위해 시간을 t0에 저장

model = RNN_Build(units)
model.compile(
    optimizer=Adam(0.001),
    loss=tf.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"],
    # experimental_run_tf_function=False,
)

model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

Epoch 1/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.5976 - loss: 0.6162
Epoch 2/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.8605 - loss: 0.3244 - val_accuracy: 0.8285 - val_loss: 0.3797
Epoch 3/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9210 - loss: 0.1995
Epoch 4/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9627 - loss: 0.1079 - val_accuracy: 0.8219 - val_loss: 0.5517


In [7]:
# 코드 7-8 모델 평가
print("훈련 데이터셋 평가...")
(loss, accuracy) = model.evaluate(train_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))
print("테스트 데이터셋 평가...")
(loss, accuracy) = model.evaluate(test_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))
t1 = time.time()
print("시간:", t1 - t0)

훈련 데이터셋 평가...
loss=0.0317, accuracy: 99.2708%
테스트 데이터셋 평가...
loss=0.5517, accuracy: 82.1875%
시간: 27.626402139663696


In [8]:
# 코드 7-9 네트워크(신경망) 구축
class RNN_Build(tf.keras.Model):
    def __init__(self, units):
        super(RNN_Build, self).__init__()
        self.embedding = layers.Embedding(
            total_words, embedding_len, input_length=max_review_len
        )

        self.rnn = Sequential(
            [
                layers.SimpleRNN(units, dropout=0.5, return_sequences=True),
                layers.SimpleRNN(units, dropout=0.5),
            ]
        )
        self.outlayer = layers.Dense(1)

    def call(self, inputs, training=None):
        x = inputs
        x = self.embedding(x)
        x = self.rnn(x)
        x = self.outlayer(x)
        prob = tf.sigmoid(x)

        return prob

In [9]:
# 코드 7-10 모델 훈련
import time

units = 64
epochs = 4
t0 = time.time()

model = RNN_Build(units)

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
    # experimental_run_tf_function=False,
)

model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

Epoch 1/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.6335 - loss: 0.6169
Epoch 2/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - accuracy: 0.8430 - loss: 0.3661 - val_accuracy: 0.8347 - val_loss: 0.3908
Epoch 3/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.8785 - loss: 0.3037
Epoch 4/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.8997 - loss: 0.2547 - val_accuracy: 0.8235 - val_loss: 0.4910


In [10]:
# 코드 7-11 모델 평가
print("훈련 데이터셋 평가...")
(loss, accuracy) = model.evaluate(train_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))
print("테스트 데이터셋 평가...")
(loss, accuracy) = model.evaluate(test_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))

t1 = time.time()
print("시간:", t1 - t0)

훈련 데이터셋 평가...
loss=0.1444, accuracy: 94.4792%
테스트 데이터셋 평가...
loss=0.4910, accuracy: 82.3518%
시간: 31.28188180923462


In [11]:
# 코드 7-12 네트워크 생성
class LSTM_Build(tf.keras.Model):
    def __init__(self, units):
        super(LSTM_Build, self).__init__()

        # 임베딩 계층
        self.embedding = tf.keras.layers.Embedding(
            total_words, embedding_len, input_length=max_review_len
        )

        # LSTM 계층
        self.lstm0 = tf.keras.layers.LSTM(units, return_sequences=True, dropout=0.5)
        self.lstm1 = tf.keras.layers.LSTM(units, return_sequences=False, dropout=0.5)

        # 출력 계층
        self.outlayer = tf.keras.layers.Dense(1)

    def call(self, inputs, training=False):
        # 임베딩 처리
        x = self.embedding(inputs)

        # 첫 번째 LSTM 계층
        x = self.lstm0(x, training=training)

        # 두 번째 LSTM 계층
        x = self.lstm1(x, training=training)

        # 출력 계층을 통과
        x = self.outlayer(x)

        # 시그모이드 확률 출력
        prob = tf.sigmoid(x)
        return prob

In [12]:
# 코드 7-13 모델 훈련
import time

units = 64
epochs = 4
t0 = time.time()

model = LSTM_Build(units)

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
    #   experimental_run_tf_function=False,
)

model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

Epoch 1/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 17s 82ms/step - accuracy: 0.6403 - loss: 0.5904
Epoch 2/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 23s 116ms/step - accuracy: 0.8666 - loss: 0.3233 - val_accuracy: 0.8385 - val_loss: 0.3707
Epoch 3/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 16s 80ms/step - accuracy: 0.8984 - loss: 0.2575
Epoch 4/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 22s 114ms/step - accuracy: 0.9200 - loss: 0.2121 - val_accuracy: 0.8342 - val_loss: 0.4054


In [13]:
# 코드 7-14 모델 평가
print("훈련 데이터셋 평가...")
(loss, accuracy) = model.evaluate(train_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
print("테스트 데이터셋 평가...")
(loss, accuracy) = model.evaluate(test_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
t1 = time.time()
print('시간:', t1-t0)

훈련 데이터셋 평가...
loss=0.1418, accuracy: 95.1923%
테스트 데이터셋 평가...
loss=0.4054, accuracy: 83.4175%
시간: 91.17441725730896


In [14]:
# 코드 7-15 네트워크 생성
class LSTM_Build(tf.keras.Model):

    def __init__(self, units):
        super(LSTM_Build, self).__init__()

        self.embedding = tf.keras.layers.Embedding(
            total_words, embedding_len, input_length=max_review_len
        )
        self.rnn = tf.keras.Sequential(
            [
                tf.keras.layers.LSTM(
                    units, dropout=0.5, return_sequences=True, unroll=True
                ),
                tf.keras.layers.LSTM(units, dropout=0.5, unroll=True),
            ]
        )
        self.outlayer = tf.keras.layers.Dense(1)

    def call(self, inputs, training=None):
        x = inputs
        x = self.embedding(x)
        x = self.rnn(x)
        x = self.outlayer(x)
        prob = tf.sigmoid(x)

        return prob

In [15]:
# 코드 7-16 모델 훈련
import time

units = 64
epochs = 4
t0 = time.time()

model = LSTM_Build(units)

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
    # experimental_run_tf_function=False,
)

model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

Epoch 1/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 25s 79ms/step - accuracy: 0.6629 - loss: 0.5801
Epoch 2/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 22s 115ms/step - accuracy: 0.8631 - loss: 0.3276 - val_accuracy: 0.8412 - val_loss: 0.3619
Epoch 3/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 15s 75ms/step - accuracy: 0.8978 - loss: 0.2577
Epoch 4/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 20s 105ms/step - accuracy: 0.9159 - loss: 0.2166 - val_accuracy: 0.8329 - val_loss: 0.3934


In [16]:
# 코드 7-17 모델 평가
print("훈련 데이터셋 평가...")
(loss, accuracy) = model.evaluate(train_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))
print("테스트 데이터셋 평가...")
(loss, accuracy) = model.evaluate(test_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))

t1 = time.time()
print("시간:", t1 - t0)

훈련 데이터셋 평가...
loss=0.1556, accuracy: 95.1402%
테스트 데이터셋 평가...
loss=0.3934, accuracy: 83.2893%
시간: 94.14238786697388


In [17]:
# 코드 7-18 네트워크 생성
class GRU_Build(tf.keras.Model):

    def __init__(self, units):
        super(GRU_Build, self).__init__()

        self.state0 = [tf.zeros([batch_size, units])]
        self.state1 = [tf.zeros([batch_size, units])]

        self.embedding = tf.keras.layers.Embedding(
            total_words, embedding_len, input_length=max_review_len
        )
        self.RNNCell0 = tf.keras.layers.GRUCell(units, dropout=0.5)
        self.RNNCell1 = tf.keras.layers.GRUCell(units, dropout=0.5)
        self.outlayer = tf.keras.layers.Dense(1)

    def call(self, inputs, training=None):
        x = inputs
        x = self.embedding(x)
        state0 = self.state0  # 초기 상태는 모두 0으로 설정
        state1 = self.state1
        for word in tf.unstack(x, axis=1):
            out0, state0 = self.RNNCell0(word, state0, training)
            out1, state1 = self.RNNCell1(out0, state1, training)

        x = self.outlayer(out1)
        prob = tf.sigmoid(x)

        return prob

In [19]:
# 코드 7-18 네트워크 생성
class GRU_Build(tf.keras.Model):

    def __init__(self, units):
        super(GRU_Build, self).__init__()

        self.state0 = [tf.zeros([batch_size, units])]
        self.state1 = [tf.zeros([batch_size, units])]

        self.embedding = tf.keras.layers.Embedding(
            total_words, embedding_len, input_length=max_review_len
        )
        self.RNN0 = tf.keras.layers.RNN(tf.keras.layers.GRUCell(units, dropout=0.5), return_sequences=True, return_state=True)
        self.RNN1 = tf.keras.layers.RNN(tf.keras.layers.GRUCell(units, dropout=0.5), return_sequences=False, return_state=True)
        self.outlayer = tf.keras.layers.Dense(1)

    def call(self, inputs, training=None):
        x = inputs
        x = self.embedding(x)
        x, state0 = self.RNN0(x, initial_state=self.state0, training=training)
        x, state1 = self.RNN1(x, initial_state=self.state1, training=training)

        x = self.outlayer(x)
        prob = tf.sigmoid(x)

        return prob

In [20]:
# 코드 7-19 모델 훈련
import time

units = 64
epochs = 4
t0 = time.time()

model = GRU_Build(units)

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
    # experimental_run_tf_function=False,
)

model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

Epoch 1/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 16s 76ms/step - accuracy: 0.6205 - loss: 0.6150
Epoch 2/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 22s 110ms/step - accuracy: 0.8583 - loss: 0.3332 - val_accuracy: 0.8438 - val_loss: 0.3576
Epoch 3/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 15s 77ms/step - accuracy: 0.8916 - loss: 0.2726
Epoch 4/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 21s 108ms/step - accuracy: 0.9161 - loss: 0.2207 - val_accuracy: 0.8393 - val_loss: 0.3943


In [21]:
# 코드 7-20 모델 평가
print("훈련 데이터셋 평가...")
(loss, accuracy) = model.evaluate(train_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))
print("테스트 데이터셋 평가...")
(loss, accuracy) = model.evaluate(test_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))
t1 = time.time()
print("시간:", t1 - t0)

훈련 데이터셋 평가...
loss=0.1495, accuracy: 95.1282%
테스트 데이터셋 평가...
loss=0.3943, accuracy: 83.9343%
시간: 86.69557738304138


In [22]:
# 코드 7-21 네트워크 생성


class GRU_Build(tf.keras.Model):

    def __init__(self, units):
        super(GRU_Build, self).__init__()

        self.embedding = tf.keras.layers.Embedding(
            total_words, embedding_len, input_length=max_review_len
        )
        self.rnn = tf.keras.Sequential(
            [
                tf.keras.layers.GRU(
                    units, dropout=0.5, return_sequences=True, unroll=True
                ),
                tf.keras.layers.GRU(units, dropout=0.5, unroll=True),
            ]
        )
        self.outlayer = tf.keras.layers.Dense(1)

    def call(self, inputs, training=None):
        x = inputs
        x = self.embedding(x)
        x = self.rnn(x)
        x = self.outlayer(x)
        prob = tf.sigmoid(x)

        return prob

In [23]:
# 코드 7-22 모델 훈련
import time

units = 64
epochs = 4
t0 = time.time()

model = GRU_Build(units)

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
    # experimental_run_tf_function=False,
)

model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

Epoch 1/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 30s 74ms/step - accuracy: 0.6393 - loss: 0.6042
Epoch 2/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 29s 151ms/step - accuracy: 0.8672 - loss: 0.3212 - val_accuracy: 0.8386 - val_loss: 0.3613
Epoch 3/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 18s 93ms/step - accuracy: 0.8949 - loss: 0.2666
Epoch 4/4
195/195 ━━━━━━━━━━━━━━━━━━━━ 26s 135ms/step - accuracy: 0.9158 - loss: 0.2236 - val_accuracy: 0.8329 - val_loss: 0.3882


In [24]:
# 코드 7-23 모델 평가
print("훈련 데이터셋 평가...")
(loss, accuracy) = model.evaluate(train_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))
print("테스트 데이터셋 평가...")
(loss, accuracy) = model.evaluate(test_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))

t1 = time.time()
print("시간:", t1 - t0)

훈련 데이터셋 평가...
loss=0.1623, accuracy: 94.7837%
테스트 데이터셋 평가...
loss=0.3882, accuracy: 83.2933%
시간: 118.12830471992493
